In [59]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import math

In [60]:
#Чтение файлов
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [61]:
def f(W,x):
    row=np.asmatrix(x)
    ans=W*row.T
    return ans

def MtrxPrint(M):
    n,m = M.shape
    for i in range(n):
        for j in range(10):
            print(M[i,j],' ',end='')
        print()
        
#def SymmetricDifferenceQuote(func,x,h):
#    return (func(x+h)-func(x-h))/2*h

In [62]:
def RegularisationL2(W):
    n,m=W.shape
    sm=0
    for i in range(n):
        for j in range(m):
            sm+=W[i,j] ** 2
    return sm

def RegularisationL2Grad(W):
    ans=W.copy()
    ans*=2
    return ans

In [63]:
def HingeLoss(W,x,y):
    ansF=f(W,x)
    sm=0
    for i in range(0,10):
        if(i!=y):
            sm+=max(0,ansF[i]-ansF[y]+1)
    return sm

def GradienAnaliticHingeLoss(W,x,y,argv1,argv2):
    grad=np.zeros((10,3072))
    grad=np.asmatrix(grad).astype(float)
    ansF=f(W,x)
    sm=0
    for i in range(10):
        if i!=y:
            if( (ansF[i]-ansF[y]+1) > 0 ):
                sm+=1
                grad[i]=x.astype(float)
    grad[y]=-x.astype(float)*sm
    return grad

def BackPropogationHingeLoss(W,x,y,argv1,argv2):
    #argv1, argv2 not used, but need to common start different types of gradients
    #forward
    scores=f(W,x)
    margin=np.zeros(10)
    reg_loss=0
    for i in range(10):
        margin[i]=max(0,scores[i]-scores[y]+1)
        if(i!=y):
            reg_loss+=margin[i]


    #backward
    #margin with max
    dMrgn = np.ones(10)*1
    count=9
    for i in range(10):
        if( (scores[i]-scores[y]+1) <= 0):
            dMrgn[i]=0
            cnt-=1
    #print(dMrgn)
    dscores=dMrgn
    dscores[y]=-dMrgn[i]*count
    
    #find dW
    dWLocal=np.zeros((10,3072))
    dWLocal=np.asmatrix(dWLocal).astype(float)
    for i in range(10):
        dWLocal[i]=x
    dW=dWLocal
    d1,d2=dW.shape
    for i in range(d1):
        dW[i]*=dscores[i]
        
    return dW

In [51]:
def SoftMaxLoss(W,x,y):
    ansF=f(W,x)
    ansF=np.exp(ansF)
    sm=np.sum(ansF)
    ans=-np.log(ansF[y]/sm)
    return ans

def GradientAnaliticSoftMaxLoss(W,x,y,argv1,argv2):
    grad=np.zeros((10,3072))
    grad=np.asmatrix(grad).astype(float)
    ansF=f(W,x)
    ansF=np.exp(ansF)
    sm=np.sum(ansF)
    for i in range(10):
        row=x.copy().astype(float)
        if (i!=y):
            row*=float(ansF[i]/sm)
        else:
            row*=float(ansF[i]/sm-1)
        grad[i]=row
    return grad

def BackPropogationSoftMax(W,x,y,argv1,argv2):
    #argv1, argv2 not used, but need to common start different types of gradients
    #forward
    scores=f(W,x)
    expScores=np.exp(scores)
    sm=np.sum(expScores)
    underLg=expScores[y]/sm;
    Loss=-np.log(underLg)
    #print(Loss)
    
    #backward
    #log(a/b) - > D(log)*D(a/b) (NOT log(a)-log(b), I understand it in next part.)
    #a/b - >  a * 1/b
    
    #DL/DL=1
    Dexp=np.zeros(10)
    Ds=np.zeros(10)
    
    DLogMinus=-1/underLg
    #print(DLogMinus)
    
    DOneDivSum=expScores[y]*DLogMinus
    DSum=-DOneDivSum/(sm*sm)
    for i in range(10):
        Dexp[i]=DSum
    Dexp[y]+=DLogMinus/sm

    for i in range(10):
        Ds[i]=np.exp(scores[i])*Dexp[i];
    
    #dW
    dWLocal=np.zeros((10,3072))
    dWLocal=np.asmatrix(dWLocal).astype(float)
    for i in range(10):
        dWLocal[i]=x
    dW=dWLocal
    d1,d2=dW.shape
    for i in range(d1):
        dW[i]*=Ds[i]
    return dW

In [10]:
def GradientNumeric(W,x,y,eps,LossFunction):
    n,m = W.shape
    grad=np.zeros((10,3072))
    grad=np.asmatrix(grad)
    lsS=LossFunction(W,x,y)
    for i in range(n):
        for j in range(m):
            Wt=W.copy()
            Wt[i,j]+=eps
            ls=LossFunction(Wt,x,y)
            grad[i,j]=(ls-lsS)/eps
    return grad

def Loss(W,X,Y,k,LossFunction):
    N = len(Y)
    sm=0
    for i in range(0,N):
        x = X[i,:]
        y = Y[i]
        sm=sm+LossFunction(W,x,y)
    return float(sm/N)+k*RegularisationL2(W)

def Gradient(W,X,Y,k,typeFunc,eps,LossFunction):
    #common gradient function in type function last two parameters are optional
    #I don't cut X_test on chunks to improve speed of learning.
    grad=np.zeros((10,3072))
    grad=np.asmatrix(grad)
    for i in range(len(Y)):
        grad+=typeFunc(W,X[i],Y[i],eps,LossFunction)
    return grad/len(Y)+k*RegularisationL2Grad(W)

In [13]:
W = np.random.rand(10,3072)
W = W*1e-6
W = np.asmatrix(W)
#test Loss on start
#print(HingeLoss(W,X_train[0],y_train[0]))     #n-1 (n=10) 
#print(SoftMaxLoss(W,X_train[0],y_train[0]))   #-ln(1/n)

[[ 2.30527127]]


In [2]:
#Compare gradients
#MtrxPrint(BackPropogationHingeLoss(W,X_train[0],y_train[0],0,0))
#print()
#MtrxPrint(GradientNumeric(W,X_train[0],y_train[0],1e-5,HingeLoss))
#print()
#MtrxPrint(GradienAnaliticHingeLoss(W,X_train[0],y_train[0],0,0))
#print(X_train[0])

In [57]:
#MtrxPrint(BackPropogationSoftMax(W,X_train[0],y_train[0],1e-5,SoftMaxLoss))
#print()
#MtrxPrint(GradientNumeric(W,X_train[0],y_train[0],1e-5,SoftMaxLoss))
#print()
#MtrxPrint(GradientAnaliticSoftMaxLoss(W,X_train[0],y_train[0],0,0))
#print(y_train[0])

In [12]:
def train(W,X,Y,k,LossFunc,GradientLoss):
    itr=0
    change=999999999
    epsMethod=1e-5
    step_size=1e-5
    before=Loss(W,X,Y,k,LossFunc)
    while (change > epsMethod and itr<1000):
        W-=step_size*Gradient(W,X,Y,k,GradientLoss,0,0)
        after=Loss(W,X,Y,k,LossFunc)
        change=abs(before-after)
        before=after
        itr+=1
    return W

def test(W,x):
    ans=f(W,x)
    mx=-9999999
    maxPos=-1
    for i in range(len(ans)):
        if ans[i]>mx:
            mx=ans[i]
            maxPos=i
    return maxPos

In [12]:
#Read data from file
dataSet = unpickle("data_batch_1")
labels=dataSet[b'labels']
data=dataSet[b'data']
filenames=dataSet[b'filenames']
MaxSize=1000 #10000 max
labels=labels[:MaxSize]
data=data[:MaxSize]
filenames=filenames[:MaxSize]

#Train, Validate, test split
X_all, X_test, y_all, y_test = train_test_split(data,labels,test_size=0.2,random_state=0)
X_train, X_validate, y_train, y_validate = train_test_split(X_all,y_all,test_size=0.2,random_state=0)

#Create matrix W
W = np.random.rand(10,3072)
W = W*1e-6
W = np.asmatrix(W)
LossFuncArr=(SoftMaxLoss,HingeLoss)
GradientFuncArr=(GradientAnaliticSoftMaxLoss,GradienAnaliticHingeLoss)

In [37]:
for L in range(2):
    LossFunc=LossFuncArr[L]
    GradientFunc=GradientFuncArr[L]
    print(LossFunc)
    for k in [0.01, 0.1, 0.5, 0.9, 1]:
        W=train(W,X_train,y_train,k,LossFunc,GradientFunc)
        good=0
        for i in range(len(X_validate)):
            ans=test(W,X_validate[i])
            if(ans == y_validate[i]):
                good+=1
        print("k =",k,"%10.5f"% (good/len(y_validate)))
    print()

<function SoftMaxLoss at 0x112ab5d90>
k = 0.01    0.25625
k = 0.1    0.32500
k = 0.5    0.32500
k = 0.9    0.31875
k = 1    0.31875

<function HingeLoss at 0x10b03fa60>
k = 0.01    0.31875
k = 0.1    0.31875
k = 0.5    0.31875
k = 0.9    0.31875
k = 1    0.31875



In [58]:
k=0.01
W=train(W,X_train,y_train,k,SoftMaxLoss,GradientAnaliticSoftMaxLoss)

NameError: name 'train' is not defined

In [64]:
good=0
for i in range(len(X_test)):
    ans=test(W,X_test[i])
    if(ans == y_test[i]):
        good+=1
print("%10.5f"% (good/len(X_test)))

NameError: name 'test' is not defined